In [1]:
import pandas as pd
import numpy as np
import healpy as hp
import h5py
import matplotlib.pyplot as plt
import yaml
import fitsio
import numba as nb
from numba import jit
import fitsio as fio

import copy

from collections import OrderedDict

import xpipe.paths as paths
import xpipe.xhandle.parbins as parbins
import xpipe.xhandle.xwrap as xwrap
import xpipe.tools.selector as selector
import xpipe.xhandle.shearops as shearops
import matplotlib as mpl
# from matplotlib import rc
mpl.rc('font',**{'family':'serif','serif':['serif']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
mpl.rc('text', usetex=True)
# mpl.rcParams['figure.dpi'] = 300

# mpl.rcParams["font.family"] = "serif"
# mpl.rcParams["mathtext.fontset"] = "dejavuserif"
import astropy.constants as constants
import astropy.units as u
import astropy.cosmology as cosmology

import scipy.interpolate as interp

cosmo = cosmology.FlatLambdaCDM(Om0=0.3, H0=70)

from importlib import reload
%config InlineBackend.figure_format = 'svg'
import pickle

import xpipe.tools.y3_sompz as sompz

import xpipe.tools.selector as sl
import xpipe.tools.visual as visual
import xpipe.xhandle.pzboost as pzboost

import sklearn
import sklearn.covariance
import sklearn.neighbors
import sklearn.decomposition
import scipy.stats as stats

import NFW
import scipy.optimize as optimize
import emcee
from cluster_toolkit import deltasigma

from cluster_toolkit import xi
from cluster_toolkit import bias


from classy import Class
from multiprocessing import Pool

import corner
paths.update_params("/home/moon/vargatn/DES/PROJECTS/xpipe/settings/params_y3lwb_meta.yml")

reading DEFAULTS from default_params.yml
updating params from: /home/moon/vargatn/DES/PROJECTS/xpipe/settings/params.yml
updating params from: /home/moon/vargatn/DES/PROJECTS/xpipe/settings/params_y3rm-stack_meta.yml
updating params from: /home/moon/vargatn/DES/PROJECTS/xpipe/settings/params_y3lwb_meta.yml

***********************

running in full mode

***********************



In [7]:
tmp = np.loadtxt("DESY3_CMASS.dat")[:, (0, 1, 6, 7)]
cmass = pd.DataFrame(data=tmp, columns=("RA", "DEC", "WSYS", "Z"))

oname = "/e/ocean1/users/vargatn/DES/DES_Y3A2_cluster/data/lenscat/LWB_DESY3_CMASS.fits"
tmp = cmass.to_records()
tmp["index"] = tmp["index"] + 1e6
# fio.write(oname, cmass.to_records())

In [8]:
tmp = np.loadtxt("DESY3_LOWZ.dat")[:, (0, 1, 6, 7)]
lowz = pd.DataFrame(data=tmp, columns=("RA", "DEC", "WSYS", "Z"))

oname = "/e/ocean1/users/vargatn/DES/DES_Y3A2_cluster/data/lenscat/LWB_DESY3_LOWZ.fits"
# fio.write(oname, lowz.to_records())

In [9]:
# tmp = cmass.to_records()

allgal = pd.concat((lowz, cmass), ignore_index=True)

oname = "/e/ocean1/users/vargatn/DES/DES_Y3A2_cluster/data/lenscat/LWB_DESY3_ALL.fits"
# fio.write(oname, allgal.to_records())

In [10]:
weights = pd.DataFrame()
weights["ww"] = allgal["WSYS"]
weights["id"] = np.arange(len(allgal))

# Processing results

In [2]:
def sigma_crit_inv(zclust, z):
    global cosmo
    prefac = (4. * np.pi * constants.G / (constants.c**2.)).to(u.pc / u.Msun)
    
    Ds = cosmo.angular_diameter_distance(z).to(u.pc)
    Dl = cosmo.angular_diameter_distance(zclust).to(u.pc)
    Dls = cosmo.angular_diameter_distance_z1z2(zclust, z).to(u.pc)
    
    val = prefac * Dl * Dls / Ds
#     print(val)
    resval = np.max((0. , val.value))
    return resval

In [3]:
main_file_path = "/e/ocean1/users/vargatn/DESY3/Y3_mastercat_03_31_20.h5"
main_file = h5py.File(main_file_path, mode = 'r')
zlows = main_file["catalog/sompz/pzdata/zlow"][:]
zhighs = main_file["catalog/sompz/pzdata/zhigh"][:]
zcens = zlows + (zhighs - zlows) / 2.
zcens = zcens[:300]
# zedges = np.concatenate((zlows[:300],  [zhighs[299],]))

source_bins = [
    main_file["catalog/sompz/pzdata/bin0"][:],
    main_file["catalog/sompz/pzdata/bin1"][:],
    main_file["catalog/sompz/pzdata/bin2"][:],
    main_file["catalog/sompz/pzdata/bin3"][:],
]

# zclust_grid = np.linspace(0.05, 1., 101)
zclust_grid_edges = np.linspace(0.00, 1.0, 101)
zclust_grid = zclust_grid_edges[:-1] + np.diff(zclust_grid_edges) / 2.

In [4]:
scritinv_tab = np.zeros(shape=(len(zclust_grid), len(zcens)))
for i, zclust in enumerate(zclust_grid):
    if i%10==0:
        print(i)
    for j, zsource in enumerate(zcens):
        scritinv_tab[i,j] = sigma_crit_inv(zclust, zsource)

0


/home/moon/vargatn/anaconda3/envs/main3/lib/python3.8/site-packages/astropy/units/quantity.py:477: RuntimeWarning: divide by zero encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


10
20
30
40
50
60
70
80
90


In [5]:
def get_bin_scinv(clust_zvals, clust_zedges, scrit_tab, source_weight):
    ccounts = np.histogram(clust_zvals, bins=clust_zedges)[0]
    zcens = clust_zedges[:-1] + np.diff(clust_zedges) / 2.
    
    scvals = np.zeros(len(zcens))
    for i, z in enumerate(zcens):
#         print(i)
        scvals[i] = np.average(scrit_tab[i], weights=source_weight)
    
    scritinv = np.average(scvals, weights=ccounts)
    return scritinv
#         scvals.append(_scvals)
    
    

In [11]:
flist, flist_jk, rlist, rlist_jk = parbins.get_file_lists(paths.params, paths.dirpaths)
# for i, clust_name in enumerate(flist_jk):
i = 0
profiles = []
for clust_name in flist_jk:
    _profiles = []
    for sbin in (1, 2, 3):
#         clust_name = flist_jk[i]
        print("processing source bin", sbin)
        clust_infos = xwrap.create_infodict(clust_name)
        clust_files = [info["outfile"].replace("_result.dat", "_bin" + str(sbin + 1) + "_result.dat") for info in clust_infos]
        clust = shearops.process_profile(clust_files, ismeta=False, weights=weights, shear=True)
        _profiles.append(clust)
    profiles.append(_profiles)    

processing source bin 1
processing source bin 2
processing source bin 3
processing source bin 1
processing source bin 2
processing source bin 3
processing source bin 1
processing source bin 2
processing source bin 3
processing source bin 1
processing source bin 2
processing source bin 3


In [12]:
# zcats = []
scinvs = []
for i in np.arange(4):
#     for j in np.arange(3):
#         print(i, j)
    _tmp = np.loadtxt("/e/ocean1/users/vargatn/DES/DES_Y3A2_cluster/data/xshear_in/lwb-y3-lowz+cmass_v01/lwb-y3-lowz+cmass_v01_y1clust_qbin-" + str(i) + ".dat")
    lenscat = pd.DataFrame(_tmp, columns=("id", "ra", "dec", "z", "dummy"))
#         infocat = pd.DataFrame(profiles[i][j].info, columns=("id", "dummy1", "dummy1"))
#         _zcat = pd.merge(infocat, lenscat, on="id", how="left")
#         zcats.append(_zcat)
    _scinvs = []
    for sbin in source_bins:
#         print("")
        _scinvs.append(get_bin_scinv(lenscat["z"], zclust_grid_edges, scritinv_tab, sbin))
    scinvs.append(_scinvs)

In [14]:
def write_profile_shear(prof, path):
    """saves DeltaSigma and covariance in text format"""

    # Saving profile
    profheader = "R [Mpc]\tgamma_t \tgamma_t_err \tgamma_x \tgamma_x_err"
    res = np.vstack((prof.rr, prof.dst, prof.dst_err, prof.dsx, prof.dsx_err)).T
    fname = path + "_profile.dat"
    print("saving:", fname)
    np.savetxt(fname, res, header=profheader)

    # Saving covariance
    np.savetxt(path + "_gt_cov.dat", prof.dst_cov)
    np.savetxt(path + "_gx_cov.dat", prof.dsx_cov)

In [15]:
tmp_profs = copy.deepcopy(profiles)
fnames = []
for i in np.arange(4):
    for s in np.arange(3):
#         scinv = scinvs[i][s+1]
        prof = tmp_profs[i][s]
#         prof.multiply(1 / scinv)
        fname = "LWB_lowz+cmass_noboost_raw_SHEAR-COLUMNS_zbin"+str(i)+"_source-bin"+str(s+1)
        fnames.append(fname)
        write_profile_shear(prof, fname)

saving: LWB_lowz+cmass_noboost_raw_SHEAR-COLUMNS_zbin0_source-bin1_profile.dat
saving: LWB_lowz+cmass_noboost_raw_SHEAR-COLUMNS_zbin0_source-bin2_profile.dat
saving: LWB_lowz+cmass_noboost_raw_SHEAR-COLUMNS_zbin0_source-bin3_profile.dat
saving: LWB_lowz+cmass_noboost_raw_SHEAR-COLUMNS_zbin1_source-bin1_profile.dat
saving: LWB_lowz+cmass_noboost_raw_SHEAR-COLUMNS_zbin1_source-bin2_profile.dat
saving: LWB_lowz+cmass_noboost_raw_SHEAR-COLUMNS_zbin1_source-bin3_profile.dat
saving: LWB_lowz+cmass_noboost_raw_SHEAR-COLUMNS_zbin2_source-bin1_profile.dat
saving: LWB_lowz+cmass_noboost_raw_SHEAR-COLUMNS_zbin2_source-bin2_profile.dat
saving: LWB_lowz+cmass_noboost_raw_SHEAR-COLUMNS_zbin2_source-bin3_profile.dat
saving: LWB_lowz+cmass_noboost_raw_SHEAR-COLUMNS_zbin3_source-bin1_profile.dat
saving: LWB_lowz+cmass_noboost_raw_SHEAR-COLUMNS_zbin3_source-bin2_profile.dat
saving: LWB_lowz+cmass_noboost_raw_SHEAR-COLUMNS_zbin3_source-bin3_profile.dat


In [ ]:
scinvs